In [16]:
import json
import argparse
import os
import subprocess
from pprint import pprint
from solc_json_parser.combined_json_parser import CombinedJsonParser
from eth_abi import encode
from eth_utils import function_abi_to_4byte_selector


def decode_abi_bin_from_compiled_json(compiled_sol):
    m = {}
    for contract_name in compiled_sol.keys():
        abi = compiled_sol.get(contract_name).get("abi")
        binary = compiled_sol.get(contract_name).get("bin")
        binary_runtime = compiled_sol.get(contract_name).get("bin-runtime")
        storage_layout = compiled_sol.get(contract_name).get("storage-layout")
        m[contract_name] = dict(
            abi=abi,
            binary_runtime=binary_runtime,
            binary=binary,
            contract=contract_name,
            storage_layout=storage_layout,
        )
    return m


def compile_file(
    file_path="contracts/state_change.sol", contract_name="TestStateChange"
):
    print ("compile file ", file_path, contract_name)
    # The input can be a file path or source code
    parser = CombinedJsonParser(file_path)
    compiler_output = parser.original_compilation_output
    # print (compiler_output)
    compiled_sol = {k.split(":")[-1]: v for k, v in compiler_output.items()}

    m_contracts = decode_abi_bin_from_compiled_json(compiled_sol)
    contract_instance = m_contracts.get(contract_name)
    contract_bin_runtime = contract_instance.get("binary_runtime")
    return contract_instance


def get_transaction_data_from_config(item, contract_instance):
    print(item)
    # print (contract_instance)
    target_function = item["function"]
    inputs = item["input"]
    print("Target function inputs ", target_function, inputs)
    # print (contract_instance['abi'])
    function_abi = [
        i
        for i in contract_instance["abi"]
        if i.get("name") == target_function and i.get("type") == "function"
    ]
    input_types = item["input_types"]
    # print (function_abi[0])
    function_abi = function_abi[0]
    four_byte = "0x" + function_abi_to_4byte_selector(function_abi).hex()
    print(four_byte)
    if len(inputs) > 0:
        encoded_data = encode(input_types, inputs).hex()
    else:
        encoded_data = ""

    return [four_byte + encoded_data]


def contruct_next_test_case(config_file, next_config):
    with open(config_file + ".out", "r") as f:
        config = json.load(f)
    # print (config)
    post_state = list(config.values())[0].get("post")[0]
    touch_state = post_state.get("touch_state")
    print("touch_state")
    print(touch_state)
    next_config_key = list(next_config.keys())[0]
    next_config[next_config_key]["pre"].update(touch_state)
    sender = next_config[next_config_key]["transaction"]["sender"]
    next_config[next_config_key]["transaction"]["nonce"] = touch_state.get(sender).get(
        "nonce"
    )
    return next_config


def run_evm(config_file, executable="../out/cpu_interpreter"):
    subprocess.run(
        [executable, "--input", config_file, "--output", config_file + ".out"]
    )


def run_test(source, config, evm_executable, output_path="/tmp/evm_test"):
    default_config = json.loads(open("configurations/default.json").read())
    print(default_config)
    # tx_sequence_list
    tx_sequence_config = json.loads(open(config).read())
    contract_name = tx_sequence_config.get("contract_name")
    contract_instance = compile_file(source, contract_name)
    contract_bin_runtime = contract_instance.get("binary_runtime")
    merged_config = []
    # the merged config fields : "env", "pre" (populated with code), "transaction" (populated with tx data and value)
    for idx, test_case in enumerate(tx_sequence_config.get("test_cases")):
        new_test = {}
        new_test["env"] = default_config["env"].copy()
        new_test["pre"] = default_config["pre"].copy()
        target_address = default_config["target_address"]
        new_test["pre"][target_address]["code"] = contract_bin_runtime
        new_test["pre"][target_address]["storage"] = tx_sequence_config.get(
            "storage", {}
        )
        new_test["transaction"] = default_config["transaction"].copy()

        new_test["transaction"]["to"] = target_address
        new_test["transaction"]["data"] = get_transaction_data_from_config(
            test_case, contract_instance
        )  # must return an array
        new_test["transaction"]["value"] = [hex(test_case.get("value", 0))]
        new_test["transaction"]["nonce"] = "0x00"
        merged_config.append({f"{contract_name}_test_{idx}": new_test})

    if not os.path.exists(output_path):
        os.makedirs(output_path)


    next_test = merged_config[0]
    print("next test ")
    print(next_test)
    for idx in range(len(merged_config)):
        # 1 test case per file
        test_name = list(next_test.keys())[0]
        # test_config = config_item.get(test_name)
        config_file = f"{output_path}/{test_name}.json"
        with open(f"{output_path}/{test_name}.json", "w") as f:
            f.write(json.dumps(next_test, indent=2))
        run_evm(config_file)
        if idx < len(merged_config) - 1:
            next_test = contruct_next_test_case(config_file, merged_config[idx + 1])

run_test("contracts/state_change.sol", "configurations/state_change.json", "../out/cpu_interpreter")

{'env': {'currentBaseFee': '0x0a', 'currentBeaconRoot': '0x0000000000000000000000000000000000000000000000000000000000000000', 'currentCoinbase': '0x2adc25665018aa1fe0e6bc666dac8fc2697ff9ba', 'currentDifficulty': '0x020000', 'currentGasLimit': '0x02540be400', 'currentNumber': '0x01', 'currentRandom': '0x0000000000000000000000000000000000000000000000000000000000020000', 'currentTimestamp': '0x03e8', 'currentWithdrawalsRoot': '0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421', 'previousHash': '0x5e20a0453cecd065ea59c37ac63e079ee08998b6045136a8ce6635c7912ec0b6'}, 'target_address': '0xcccccccccccccccccccccccccccccccccccccccc', 'pre': {'0xa94f5374fce5edbc8e2a8697c15331677e6ebf0b': {'balance': '0x0ba1a9ce0ba1a9ce', 'code': '0x', 'nonce': '0x00', 'storage': {}}, '0xcccccccccccccccccccccccccccccccccccccccc': {'balance': '0x0ba1a9ce0ba1a9ce', 'code': '0x', 'nonce': '0x00', 'storage': {}}}, 'transaction': {'data': ['0x00'], 'gasLimit': ['0x04c4b400'], 'gasPrice': '0x0a', 'nonce'